RUNNA QUESTO CODICE PER PROVARE A DISEGNARE!

In [3]:
import numpy as np
import tkinter as tk
from tkinter import Canvas, Button, Label, Frame
from PIL import Image, ImageDraw, ImageOps
from tensorflow.keras.models import load_model
import requests
from io import BytesIO
import tempfile

# Mappa delle classi
classes = {0: 'airplane', 1: 'apple', 2: 'banana', 3: 'cactus', 4: 'car', 5: 'clock', 6: 'cloud', 7: 'door',
           8: 'eye', 9: 'fish', 10: 'fork', 11: 'ice cream', 12: 'line', 13: 'lollipop', 14: 'octopus', 15: 'pencil',
           16: 'smiley face', 17: 'star', 18: 'sun', 19: 'umbrella'}

github_model_url = "https://github.com/tommasosenatori/ML_Real-Time-Sketch-Recognition/raw/main/GUI/QuickDraw!.h5"

# Carico il modello addestrato da GitHub
response = requests.get(github_model_url)
model_path = tempfile.NamedTemporaryFile(delete=False, suffix=".h5")
model_path.write(response.content)
model_path.close()

model = load_model(model_path.name)



# Creazione dell'interfaccia grafica con Tkinter

# Creo una window
window = tk.Tk()
window.title("Real-Time Digit Recognition")

# Dimensioni canvas (+ grande così disgenare è + facile)
canvas_width = 280
canvas_height = 280

# Creo canvas
canvas = Canvas(window, width=canvas_width, height=canvas_height, bg='white')
canvas.pack()


# Display della predizione
label = Label(window, text="Prediction: None", font=("Helvetica", 16))
label.pack()

# Frame della tabella delle predizioni
prediction_frame = Frame(window)
prediction_frame.pack()

# Clear function (resetta il canvas, anche le predizioni)
def clear_canvas(canvas):
    canvas.delete("all")
    label.config(text="Prediction: None")
    update_prediction_table(None)  # Clear the prediction table

# Function di update della prediction table: stampa la top 10 delle predizioni
def update_prediction_table(predictions, num_classes=10):
    for widget in prediction_frame.winfo_children():
        widget.destroy()

    if predictions is not None:
        # Sort predictions in ordine decrescente
        sorted_indices = np.argsort(predictions)[::-1]

        # Display the top N predictions
        for i in range(num_classes):
            class_index = sorted_indices[i]
            class_name = classes[class_index]
            probability = predictions[class_index]
            text = f"{class_name}: {probability * 100:.2f}%"
            prediction_label = Label(prediction_frame, text=text, font=("Helvetica", 12))
            prediction_label.pack()


# Function di predict della classe giusta
def predict_drawing():
    # Salvo il contenuto del canvas come un'immagine
    canvas.postscript(file="tmp.eps", colormode="mono")
    img = Image.open("tmp.eps")

    # Converto in grayscale
    img = img.convert("L")

    # Resize a 28x28 pixels, così img è interpretabile dal modello
    img = img.resize((28, 28), Image.LANCZOS)

    # Inverto img (sfondo nero, disegno bianco)
    img = ImageOps.invert(img)

    # Flattening e normalization
    img_array = np.array(img).reshape(1, 784) / 255.0

    # Prediction vera e propria
    prediction = model.predict(img_array)
    predicted_class_index = np.argmax(prediction)
    predicted_class_name = classes[predicted_class_index]

    # Aggiorno tabella
    label.config(text=f"Prediction: {predicted_class_name}")

    update_prediction_table(prediction[0])

# Variabili che indicano se l'user sta disegnando, o se la gomma è attiva
drawing = False
eraser = False

# Function che si attiva se l'user clicca o tiene premuto col mouse
def start_drawing(event):
    global drawing
    drawing = True
    x, y = event.x, event.y
    if eraser:
        canvas.create_rectangle(x - 10, y - 10, x + 10, y + 10, fill='white', outline='white') # per gomma
    else:
        canvas.create_oval(x - 5, y - 5, x + 5, y + 5, fill='black') # per matita

# Function che si attiva se l'user muove il mouse mentre tiene premuto
def draw(event):
    if drawing:
        x, y = event.x, event.y
        if eraser:
            canvas.create_rectangle(x - 10, y - 10, x + 10, y + 10, fill='white', outline='white')
        else:
            canvas.create_oval(x - 5, y - 5, x + 5, y + 5, fill='black')

# Function per quando l'user rilascia il mouse, e printa la predizione
def stop_drawing(event):
    global drawing
    drawing = False
    predict_drawing()

# Function della gomma (ON/OFF)
def toggle_eraser():
    global eraser
    eraser = not eraser
    eraser_button.config(text="Eraser On" if eraser else "Eraser Off")

# Bind (associo) il tasto sx (B1) del mouse al disegnare
canvas.bind("<Button-1>", start_drawing)
canvas.bind("<B1-Motion>", draw)
canvas.bind("<ButtonRelease-1>", stop_drawing)

# Creo clear button
clear_button = Button(window, text="Clear", command=lambda: clear_canvas(canvas))
clear_button.pack()

# Creo toggle eraser button
eraser_button = Button(window, text="Eraser Off", command=toggle_eraser)
eraser_button.pack()

window.mainloop() # il main di Tkinter



1/1 [==============================] - 0s 22ms/step
